## Import dependencies

In [23]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

## Prepare data for BayesNet
### Final Cleaning

In [7]:
data_file = "data/all_sc.tsv"

# Load the file and do some preliminary cleaning
df = pd.read_table(data_file)
df = df.set_index(["File Name","scan"])
df = df.rename(columns={"precursor_intenisty": "precursor_intensity"})
df = df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

# Rename to be more descriptive
df = df.rename(columns={"Base Peak Intensity": "Max Spectrum Intensity"})

# Convert the m/z array from a string to a list
df["mz_array"] = df["mz_array"].str.strip("[]").str.split(", ") # Convert to list
df["mz_array"] = df["mz_array"].map(lambda l: [float(e) for e in l])  # Converts each element to a float instead of string

# Compute the max spectrum m/z and min spectrum  m/z ratios
df["Max Spectrum Mass-To-Charge Ratio"] = df["mz_array"].map(max)
df["Min Spectrum Mass-To-Charge Ratio"] = df["mz_array"].map(min)

# Extract the ions from the rows
df_series = df.drop(columns=["Matched Ion Intensities","Matched Ion Mass-To-Charge Ratios"])
df_intensities = df[["Matched Ion Intensities"]].copy()
df_mz = df[["Matched Ion Mass-To-Charge Ratios"]].copy()

# Deal with getting the Ion Name, Ion Charge, Ion Number (with respect to the peptide)
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(";")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(", ")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.strip("[]")
df_series = df_series.rename(columns={"Matched Ion Series": "Ion Name"})
df_series[["Ion Type", "Ion Number", "Ion Charge"]] = df_series["Ion Name"].str.extract(r"([abcxyz])(\d{1,3})([+-]\d)") # Tried to handle every possible case here, but this may need to be amended
df_series = df_series.astype({"Ion Charge": int, "Ion Number": int})
df_series = df_series.set_index("Ion Name", append=True)

# Deal with getting the Ion Intensity
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(";")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(", ")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.strip("[]")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(":")
df_intensities = pd.DataFrame(df_intensities["Matched Ion Intensities"].to_list(), columns=["Ion Name", "Intensity"], index=df_intensities.index)
df_intensities = df_intensities.set_index("Ion Name", append=True)
df_intensities = df_intensities.astype({"Intensity": float})

# Deal with getting the Ion m/z
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(";")
df_mz = df_mz.explode("Matched Ion Mass-To-Charge Ratios")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(", ")
df_mz = df_mz.explode("Matched Ion Mass-To-Charge Ratios")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.strip("[]")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(":")
df_mz = pd.DataFrame(df_mz["Matched Ion Mass-To-Charge Ratios"].to_list(), columns=["Ion Name", "Mass-To-Charge Ratio"], index=df_mz.index)
df_mz = df_mz.set_index("Ion Name", append=True)
df_mz = df_mz.astype({"Mass-To-Charge Ratio": float})

# Combine them together
df_clean = df_series.join(df_intensities, how="inner")
df_clean = df_clean.join(df_mz, how="inner")

# df_clean = df_clean[["Ion Type", "Ion Number", "Ion Charge", "Mass-To-Charge Ratio", "Intensity", "Base Peak Intensity", "mz_array", "intensity_array", "precursor_intensity", "peptide"]]

df_clean

Matched Ion Counts    QValue  \
File Name              scan  Ion Name                                 
D19_15um30cm_SC1-calib 13311 b1+1                      23  0.000000   
                             b2+1                      23  0.000000   
                             b4+1                      23  0.000000   
                             b5+1                      23  0.000000   
                             b6+1                      23  0.000000   
...                                                   ...       ...   
D19_15um30cm_SC5-calib 23659 y2+1                       6  0.009988   
                             y5+1                       6  0.009988   
                             y6+1                       6  0.009988   
                             y8+1                       6  0.009988   
                             b3+1                       6  0.009988   

                                                                                 peptide  \
File Name              scan  Ion Name                                                      
D19_15um30cm_SC1-calib 13311 b1+1      RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...   
                             b2+1      RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...   
                             b4+1      RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...   
                             b5+1      RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...   
                             b6+1      RAPSVANVGSHC[Common Fixed:Carbamidomethyl on C...   
...                                                                                  ...   
D19_15um30cm_SC5-calib 23659 y2+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...   
                             y5+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...   
                             y6+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...   
                             y8+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...   
                             b3+1      C[Common Fixed:Carbamidomethyl on C]GDLEEELKNV...   

                                      Previous Amino Acid Next Amino Acid  \
File Name              scan  Ion Name                                       
D19_15um30cm_SC1-calib 13311 b1+1                       R               I   
                             b2+1                       R               I   
                             b4+1                       R               I   
                             b5+1                       R               I   
                             b6+1                       R               I   
...                                                   ...             ...   
D19_15um30cm_SC5-calib 23659 y2+1                       K               S   
                             y5+1                       K               S   
                             y6+1                       K               S   
                             y8+1                       K               S   
                             b3+1                       K               S   

                                       Max Spectrum Intensity  \
File Name              scan  Ion Name                           
D19_15um30cm_SC1-calib 13311 b1+1                 4185.303711   
                             b2+1                 4185.303711   
                             b4+1                 4185.303711   
                             b5+1                 4185.303711   
                             b6+1                 4185.303711   
...                                                       ...   
D19_15um30cm_SC5-calib 23659 y2+1                 1418.214111   
                             y5+1                 1418.214111   
                             y6+1                 1418.214111   
                             y8+1                 1418.214111   
                             b3+1                 1418.214111   

                                                                      

### Binning the Mass-to-Charge Ratios and Intensities

In [8]:
# Define the number of bins
num_sector_bins = 3
num_intensity_bins = 3

# Perform the binning
df_clean["Sector"] = ((df_clean["Mass-To-Charge Ratio"] - df_clean["Min Spectrum Mass-To-Charge Ratio"]) / (df_clean["Max Spectrum Mass-To-Charge Ratio"] - df_clean["Min Spectrum Mass-To-Charge Ratio"]) * num_sector_bins).astype(int)
df_clean.loc[df_clean["Sector"] == num_sector_bins, "Sector"] = num_sector_bins - 1 # Overwrite the ones that are on the edge to be the correct bin
df_clean["Intensity"] = ((df_clean["Intensity"] / df_clean["Max Spectrum Intensity"]) * num_intensity_bins).astype(int)
df_clean.loc[df_clean["Intensity"] == num_intensity_bins, "Intensity"] = num_intensity_bins - 1 # Overwrite the ones that are on the edge to be the correct bin

# Filter out everything else
df_clean = df_clean[["Ion Type", "Intensity", "Sector"]]

df_clean

Ion Type  Intensity  Sector
File Name              scan  Ion Name                            
D19_15um30cm_SC1-calib 13311 b1+1            b          0       0
                             b2+1            b          0       0
                             b4+1            b          0       0
                             b5+1            b          1       0
                             b6+1            b          1       0
...                                        ...        ...     ...
D19_15um30cm_SC5-calib 23659 y2+1            y          0       0
                             y5+1            y          1       0
                             y6+1            y          0       1
                             y8+1            y          1       1
                             b3+1            b          2       0

[194045 rows x 3 columns]

## Implement BayesNet

In [9]:
# Instantiate
net = BayesianNetwork()

# Construct
net.add_nodes_from(["Ion Type", "Intensity", "Sector"])
net.add_edges_from([("Ion Type", "Intensity"), ("Sector", "Intensity")])

# Train
net.fit(df_clean)

## Get Probability Tables

We want to be able to take a new ion with its m/z ratio $M$ and intensity $I$ and determine the probability of it being a certain type, $T$. I.e. $P(T | M,I)$

In [10]:
cpds = net.get_cpds()
intensity_table = cpds[1]

intensity_table.assignment([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
intensity_table.get_state_no("Intensity", 2)

2

In [25]:
intensity_table.assignment([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

[[('Intensity', 0), ('Ion Type', 'b'), ('Sector', 0)],
 [('Intensity', 0), ('Ion Type', 'b'), ('Sector', 1)],
 [('Intensity', 0), ('Ion Type', 'b'), ('Sector', 2)],
 [('Intensity', 0), ('Ion Type', 'y'), ('Sector', 0)],
 [('Intensity', 0), ('Ion Type', 'y'), ('Sector', 1)],
 [('Intensity', 0), ('Ion Type', 'y'), ('Sector', 2)],
 [('Intensity', 1), ('Ion Type', 'b'), ('Sector', 0)],
 [('Intensity', 1), ('Ion Type', 'b'), ('Sector', 1)],
 [('Intensity', 1), ('Ion Type', 'b'), ('Sector', 2)],
 [('Intensity', 1), ('Ion Type', 'y'), ('Sector', 0)],
 [('Intensity', 1), ('Ion Type', 'y'), ('Sector', 1)],
 [('Intensity', 1), ('Ion Type', 'y'), ('Sector', 2)],
 [('Intensity', 2), ('Ion Type', 'b'), ('Sector', 0)],
 [('Intensity', 2), ('Ion Type', 'b'), ('Sector', 1)],
 [('Intensity', 2), ('Ion Type', 'b'), ('Sector', 2)],
 [('Intensity', 2), ('Ion Type', 'y'), ('Sector', 0)],
 [('Intensity', 2), ('Ion Type', 'y'), ('Sector', 1)],
 [('Intensity', 2), ('Ion Type', 'y'), ('Sector', 2)]]

In [27]:
intensity_table.get_values().flatten()

array([0.75441506, 0.83752985, 0.8426139 , 0.84530654, 0.67414151,
       0.61730764, 0.17035624, 0.11384849, 0.08099402, 0.11610382,
       0.21112997, 0.23138862, 0.0752287 , 0.04862166, 0.07639208,
       0.03858964, 0.11472852, 0.15130374])

1.0